In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 21

In [4]:
name = list[k].getName()
name

'RP60'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.5f, %.5f] " % ( resultsMC[4], resultsMC[5])


b = "[%.5f, %.5f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.5f, %.5f] " % ( resultSubset[4], resultSubset[5])


In [8]:
mc1 = np.format_float_scientific(np.float32(resultsMC[4]))
mc2 = np.format_float_scientific(np.float32(resultsMC[5]))

ic1 = "["+mc1+", "+mc2+"]"
ic1

'[3.6691397e-02, 5.4560963e-02]'

In [9]:
formis1 = np.format_float_scientific(np.float32(resultFORMIS[4]))
formis2 = np.format_float_scientific(np.float32(resultFORMIS[5]))

ic2 = "["+formis1+", "+formis2+"]"
ic2

'[4.355566e-02, 6.471579e-02]'

In [10]:
subset1 = np.format_float_scientific(np.float32(resultSubset[4]))
subset2 = np.format_float_scientific(np.float32(resultSubset[5]))

ic3 = "["+subset1+", "+subset2+"]"

ic3

'[4.4358473e-02, 4.8201527e-02]'

In [11]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.03669, 0.05456] ', None],
 [None, None, None, '[0.03669, 0.05456] ', None],
 [None, None, None, '[0.03669, 0.05456] ', None],
 [None, None, None, '[0.03669, 0.05456] ', None],
 [None, None, None, '[0.03669, 0.05456] ', None],
 [None, None, None, '[0.03669, 0.05456] ', None]]

In [12]:
l = np.array([[1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a],[1, 1, 1, ic3, 1, a]])

In [13]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité de référence de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  "%.2f " % (result[i][2])


results[0][3] = ic1
results[1][3] = "-----------------"
results[2][3] = "-----------------"
results[3][3] = ic2
results[4][3] = ic3


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2,x3,x4,x5]->[var g1 := x1 - x5;var g2 := x2 - x5 / 2;var g3 := x3 - x5 / 2;var g4 := x4 - x5 / 2;var g5 := x2 - x5;var g6 := x3 - x5;var g7 := x4 - x5;var g8 := min(g5, g6);var g9 := max(g7, g8);var g10 := min(g2, g3, g4);var g11 := max(g10, g9);gsys := min(g1, g11)]


,Probabilité de référence de RP60,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,4.6e-02,4.562618e-02,3.7382002e-04,"[3.6691397e-02, 5.4560963e-02]",3178,2.09
FORM,4.6e-02,4.4839762e-02,1.1602384e-03,-----------------,100,1.60
SORM,4.6e-02,4.483973e-02,1.1602709e-03,-----------------,161,1.60
FORM-IS,4.6e-02,5.413572e-02,8.135721e-03,"[4.355566e-02, 6.471579e-02]",252,0.75
SUBSET,4.6e-02,4.628e-02,2.8e-04,"[4.4358473e-02, 4.8201527e-02]",20000,2.22


In [15]:
df.to_csv('problem60.csv')